### Importing the libraries

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
nlp = spacy.load('en_core_web_sm')

### Importing the training data

In [3]:
train_data = pd.read_csv('train.csv')

In [4]:
train_data.shape

(20800, 5)

In [5]:
train_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


### Data Cleaning

In [6]:
train_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
train_data = train_data.fillna('')

In [8]:
train_data['new'] = train_data["title"]+' '+train_data['author']

In [9]:
train_data.new[0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It Darrell Lucus'

In [10]:
def preprocessing(data):
    doc = nlp(data)
    clean_text = [str(token.lemma_) for token in doc if not token.is_punct and not token.is_stop and not token.is_digit]
    return ' '.join(clean_text)

In [11]:
train_data['new'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It Darrell Lucus'

In [12]:
train_data['new'] = train_data['new'].apply(preprocessing)

In [13]:
train_data['new'][0]

'House Dem Aide Comey Letter Jason Chaffetz tweet Darrell Lucus'

In [14]:
X_train = train_data['new'].values
y_train = train_data['label'].values

In [15]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X_train)

X_train = vectorizer.transform(X_train)

### Model training

In [16]:
model = LogisticRegression()

In [17]:
model.fit(X_train, y_train)

LogisticRegression()

In [18]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, y_train)
training_data_accuracy

0.9890865384615385

### Prediction of the Test data

In [19]:
test_data = pd.read_csv('test.csv')

In [20]:
test_data.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [21]:
test_data = test_data.fillna('')

In [22]:
test_data['new'] = test_data["title"]+' '+test_data['author']

In [23]:
test_data['new'] = test_data['new'].apply(preprocessing)

In [24]:
X_test = test_data['new'].values

In [25]:

X_test = vectorizer.transform(X_test)

In [26]:
y_pred = model.predict(X_test)

In [27]:
test_data.columns

Index(['id', 'title', 'author', 'text', 'new'], dtype='object')

In [28]:
output = pd.DataFrame({'id': test_data.id, 'label': y_pred})
output.to_csv('my_submission.csv', index=False)

In [29]:
# Score --> 0.98525
# https://www.kaggle.com/c/fake-news/